In [1]:
import torch
import pandas as pd
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
import torchvision.models as models
import torchvision.transforms
import torchaudio
import torchaudio.transforms as T
import librosa

from torchvision.models import ResNet

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [23]:
sig,_ = librosa.load("D:/project/voice_pathology_ai/voice_data/fusion/healthy/phrase/1-phrase.wav",sr=16000)

In [2]:
import opensmile

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)
y = smile.process_file("D:/project/voice_pathology_ai/voice_data/fusion/healthy/phrase/1-phrase.wav")

In [24]:
sig

array([0.00062774, 0.00136639, 0.00109393, ..., 0.00089322, 0.00074682,
       0.        ], dtype=float32)

In [25]:
handcrafted = smile.process_signal(
                sig,
                16000
            )

In [29]:
torch.from_numpy(handcrafted.to_numpy()).size()

torch.Size([1, 6373])

In [3]:
y.reset_index(drop=True)

,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,audspec_lengthL1norm_sma_percentile1.0,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,2.271744,0.255102,0.0,0.399389,0.665472,1.073285,0.266083,0.407812,0.673896,0.080488,...,9.021909,0.544175,3.330909,3.4473,-15.953115,0.439667,125.111931,61.925774,105.369995,57.428726


In [4]:
pip install --upgrade numpy --user

Note: you may need to restart the kernel to use updated packages.


In [18]:
from disvoice.glottal import Glottal
glottal=Glottal()
file_audio="D:/project/voice_pathology_ai/voice_data/fusion/healthy/phrase/1-phrase.wav"
features=glottal.extract_features_file(file_audio, static=False, plots=False, fmt="dataframe")

In [19]:
features.columns

Index(['var GCI', 'avg NAQ', 'std NAQ', 'avg QOQ', 'std QOQ', 'avg H1H2',
       'std H1H2', 'avg HRF', 'std HRF'],
      dtype='object')

In [3]:
pip uninstall disvoice

In [4]:
pip install "numpy<1.21.0" --user

Note: you may need to restart the kernel to use updated packages.
